In [319]:
# %load "../batchedRNN/newTrainUtils.py"
import logging, sys
import torch
import h5py
import os
import numpy as np
import torch.utils.data as torchUtils
import torch.optim as optim
from functools import partial
import torch.nn as nn
import json
from shutil import copy2, copyfile, copytree
import argparse

logging.basicConfig(stream=sys.stderr,level=logging.DEBUG)

def getSaveDir():
    saveDir = '../save/models/model0/'
    while os.path.isdir(saveDir):
        numStart = saveDir.rfind("model")+5
        numEnd = saveDir.rfind("/")
        saveDir = saveDir[:numStart] + str(int(saveDir[numStart:numEnd])+1) + "/"
    os.mkdir(saveDir)
    return saveDir

def saveUsefulData():
    argsFile = args.save_dir + "args.txt"
    with open(argsFile, "w") as f:
        f.write(json.dumps(vars(args)))
    copy2("./train.py", args.save_dir+"train.py")
    copy2("./utils.py", args.save_dir+"utils.py")
    copy2("./gridSearchOptimize.py", args.save_dir+"gridsearchOptimize.py")
    copytree("./model", args.save_dir+"model/")

def getLoaderAndScaler(dataDir, category):
    logging.info("Getting {} loader".format(category))
    f = np.load(os.path.join(dataDir, category + '.npz'))
    my_dataset = torchUtils.TensorDataset(torch.Tensor(f["inputs"]),torch.Tensor(f["targets"])) # create your datset
    scaler = getScaler(f["inputs"])
    sequence_len = f['inputs'].shape[1]
    x_dim = f['inputs'].shape[2]
    channels = f["inputs"].shape[3]
    shf = False
    if category == "train":
        shf = True
    loader = torchUtils.DataLoader(
        my_dataset,
        batch_size=args.batch_size,
        shuffle=shf,
        num_workers=0,
        pin_memory=False,
        drop_last=True
        )
    return loader, scaler, sequence_len, x_dim, channels # create your dataloader

def getDataLoaders(dataDir, debug=False):
    loaders = {}
    logging.info("Getting loaders")
    if debug:
        categories = ["test"]
        scalerSet = "test"
    else:
        categories = ["train", "val", "test"]
        scalerSet = "train"
    for category in categories:
        loader, scaler, sequence_len, x_dim, channels = getLoaderAndScaler(dataDir, category)
        if category == scalerSet:
            loaders["scaler"] = scaler
            loaders["sequence_len"] = sequence_len
            loaders["x_dim"] = x_dim
            loaders["channels"] = channels
        loaders[category] = loader
    return loaders

def transformBatch(batch, scaler=None):
    x = scaler.transform(batch[0]).permute(1,0,3,2)
    y = scaler.transform(batch[1])[...,0].permute(1,0,2)
    return x, y

class StandardScaler:
    """
    Standard the input
    """

    def __init__(self, mean0, std0, mean1=0, std1=1):
        self.mean0 = mean0
        self.mean1 = mean1
        self.std0 = std0
        self.std1 = std1

    def transform(self, data):
        mean = torch.zeros(data.size())
        mean[...,0] = self.mean0
        mean[...,1] = self.mean1
        std = torch.ones(data.size())
        std[...,0] = self.std0
        std[...,1] = self.std1
        return torch.div(torch.sub(data,mean),std)

    def inverse_transform(self, data):
        """
        Inverse transform is applied to output and target.
        These are only the speeds, so only use the first 
        """
        mean = torch.ones(data.size()) * self.mean0
        std = torch.ones(data.size()) * self.std0
        transformed = torch.add(torch.mul(data, std), mean)
        return transformed.permute(1,0,2)

    def inverse_transform_both_layers(self, data):
        mean = torch.zeros(data.size())
        mean[...,0] = self.mean0
        mean[...,1] = self.mean1
        std = torch.ones(data.size())
        std[...,0] = self.std0
        std[...,1] = self.std1
        transformed =  torch.add(torch.mul(data, std), mean)
        return transformed.permute(1,0,3,2)

def getScaler(trainX):
    mean = np.mean(trainX[...,0])
    std = np.std(trainX[...,0])
    return StandardScaler(mean, std)

def getLoss(output, target, scaler):
    output = scaler.inverse_transform(output)
    target = scaler.inverse_transform(target)
    assert output.size() == target.size(), "output size: {}, target size: {}".format(output.size(), target.size())
    criterion = "RMSE"
    if criterion == "RMSE":
        criterion = nn.MSELoss()
        return torch.sqrt(criterion(output, target))
    elif criterion == "L1Loss":
        criterion = nn.L1Loss()
        return criterion(output, target)
    else:
        assert False, "bad loss function"



In [241]:
speeds = (np.random.randn(5, 4,3)* 10) + 65
timeOfDay = np.random.rand(5, 4,3)
print(speeds)
print(timeOfDay)
inputData = np.stack((speeds, timeOfDay), axis=3)

[[[63.99346889 47.31658719 66.478928  ]
  [83.58477845 68.14084886 78.80703529]
  [67.64627774 74.17082465 67.442546  ]
  [69.3523733  67.5303261  81.86291422]]

 [[61.54254788 80.98376953 57.65292898]
  [70.39640119 69.16869407 61.88037896]
  [71.85493607 52.49022008 76.247361  ]
  [70.47420001 65.06524411 62.03843337]]

 [[62.03218409 59.03003066 89.33351477]
  [72.94890947 53.5682124  79.24120003]
  [62.53528751 60.79562866 45.38959797]
  [86.76727584 75.95259221 69.4931112 ]]

 [[65.6605624  55.51503742 62.93440345]
  [48.51410024 44.77698867 66.58609757]
  [67.99575007 63.9415507  44.65073071]
  [74.10958984 54.92022061 41.04563709]]

 [[64.22282472 76.70059989 58.24453939]
  [49.52074848 57.83752356 78.99927515]
  [67.90346461 72.69957004 67.06095684]
  [79.09172829 67.20691796 56.98543221]]]
[[[0.87418004 0.24175163 0.42537499]
  [0.38419471 0.76271611 0.17384757]
  [0.28060401 0.53252047 0.94650913]
  [0.48380768 0.99622422 0.57556417]]

 [[0.44128815 0.89676788 0.99854488]
  [

In [242]:
inputData.shape

(5, 4, 3, 2)

In [243]:
inputData

array([[[[6.39934689e+01, 8.74180041e-01],
         [4.73165872e+01, 2.41751631e-01],
         [6.64789280e+01, 4.25374988e-01]],

        [[8.35847784e+01, 3.84194710e-01],
         [6.81408489e+01, 7.62716106e-01],
         [7.88070353e+01, 1.73847569e-01]],

        [[6.76462777e+01, 2.80604010e-01],
         [7.41708247e+01, 5.32520465e-01],
         [6.74425460e+01, 9.46509135e-01]],

        [[6.93523733e+01, 4.83807678e-01],
         [6.75303261e+01, 9.96224224e-01],
         [8.18629142e+01, 5.75564173e-01]]],


       [[[6.15425479e+01, 4.41288150e-01],
         [8.09837695e+01, 8.96767879e-01],
         [5.76529290e+01, 9.98544885e-01]],

        [[7.03964012e+01, 4.15575004e-01],
         [6.91686941e+01, 8.44708266e-01],
         [6.18803790e+01, 4.90519910e-01]],

        [[7.18549361e+01, 8.88610682e-01],
         [5.24902201e+01, 5.72517988e-01],
         [7.62473610e+01, 5.30902124e-01]],

        [[7.04742000e+01, 7.43712089e-02],
         [6.50652441e+01, 7.17212344e-

In [244]:
speedsTarget = (np.random.randn(5,4,3)* 10) + 65
timeOfDayTarget = np.random.rand(5,4,3)
targetData = np.stack((speedsTarget, timeOfDayTarget), axis=3)
print(targetData)

[[[[6.15396237e+01 2.03808093e-01]
   [8.25922890e+01 1.01084024e-01]
   [6.28614131e+01 4.56517006e-01]]

  [[6.29506740e+01 4.01087303e-02]
   [6.51122754e+01 3.66117226e-01]
   [5.55373589e+01 2.09935747e-01]]

  [[7.91902415e+01 7.16083580e-01]
   [7.74048880e+01 1.55617038e-01]
   [6.22697363e+01 2.17075447e-01]]

  [[5.23740140e+01 8.17640963e-01]
   [6.81767309e+01 7.22203069e-01]
   [7.41631781e+01 4.71626672e-02]]]


 [[[5.51807099e+01 4.73687388e-01]
   [5.55771967e+01 3.20351286e-01]
   [6.97079219e+01 6.08801304e-01]]

  [[6.46319291e+01 5.60634988e-01]
   [7.98639943e+01 1.76793705e-01]
   [5.75901350e+01 4.12946621e-01]]

  [[7.67450552e+01 8.01613975e-01]
   [8.00392231e+01 2.99270062e-01]
   [7.13157318e+01 3.95089207e-01]]

  [[6.78573485e+01 3.73293435e-01]
   [6.46762864e+01 2.35085112e-01]
   [7.18587422e+01 4.80692518e-01]]]


 [[[6.98818010e+01 1.10009563e-01]
   [5.98981697e+01 2.76521346e-01]
   [6.94314512e+01 6.82984574e-01]]

  [[7.37983672e+01 3.27705275e-01

In [245]:
scaler = getScaler(inputData, targetData)

In [246]:
scaler.mean0

65.6722303113465

In [247]:
scaler.std0

10.811694913373284

In [248]:
scaler.mean1

0

In [249]:
scaler.std1

1

In [250]:
inputTTransformed, targetTTransformed = transformBatch([torch.FloatTensor(inputData), torch.FloatTensor(targetData)], scaler)

In [251]:
inputTTransformed.size()

torch.Size([4, 5, 2, 3])

In [252]:
print(np.mean(inputTTransformed[:,:,0,:].numpy()), np.std(inputTTransformed[:,:,0,:].numpy()))

-3.6358833e-07 1.0


In [253]:
targetTTransformed.size()

torch.Size([4, 5, 3])

In [254]:
print(np.mean(targetTTransformed.numpy()), np.std(targetTTransformed.numpy()))

-0.04899164 0.9387398


In [255]:
targetTTransformed.size()

torch.Size([4, 5, 3])

In [256]:
print(np.mean(np.concatenate((inputTTransformed.numpy()[:,:,0,:], targetTTransformed.numpy()),axis=1)),np.std(np.concatenate((inputTTransformed.numpy()[:,:,0,:], targetTTransformed.numpy()),axis=1)))

-0.024495991 0.970163


In [257]:
np.mean(inputData[...,0])

65.6722303113465

In [258]:
scaler.transform(torch.FloatTensor(inputData))[...,0].mean()

tensor(-0.0000)

In [259]:
inputData[0,:,:,0]

array([[63.99346889, 47.31658719, 66.478928  ],
       [83.58477845, 68.14084886, 78.80703529],
       [67.64627774, 74.17082465, 67.442546  ],
       [69.3523733 , 67.5303261 , 81.86291422]])

In [260]:
inputTTransformed[:,0,0,:]

tensor([[-0.1553, -1.6978,  0.0746],
        [ 1.6568,  0.2283,  1.2149],
        [ 0.1826,  0.7861,  0.1637],
        [ 0.3404,  0.1719,  1.4975]])

In [261]:
scaler.mean0

65.6722303113465

In [262]:
inputData.shape

(5, 4, 3, 2)

In [263]:
targetTTransformed.shape

torch.Size([4, 5, 3])

In [283]:
inputData.shape

(5, 4, 3, 2)

In [295]:
data = []
data.append(torch.FloatTensor(inputData))
data.append(torch.FloatTensor(targetData))
transedX, transedY = transformBatch(data, scaler)
print(transedX.shape)
print(transedY.shape)

torch.Size([4, 5, 2, 3])
torch.Size([4, 5, 3])


In [296]:
mockOutput = transedX[:,:,0,:]

In [297]:
invTransOutput = scaler.inverse_transform(mockOutput)

In [298]:
invTransOutput.shape

torch.Size([5, 4, 3])

In [299]:
inputData.shape

(5, 4, 3, 2)

In [300]:
invTransOutput[0]

tensor([[63.9935, 47.3166, 66.4789],
        [83.5848, 68.1408, 78.8070],
        [67.6463, 74.1708, 67.4425],
        [69.3524, 67.5303, 81.8629]])

In [301]:
inputData[0,:,:,0]

array([[63.99346889, 47.31658719, 66.478928  ],
       [83.58477845, 68.14084886, 78.80703529],
       [67.64627774, 74.17082465, 67.442546  ],
       [69.3523733 , 67.5303261 , 81.86291422]])

In [305]:
scaler.inverse_transform(transedY)[0]

tensor([[61.5396, 82.5923, 62.8614],
        [62.9507, 65.1123, 55.5374],
        [79.1902, 77.4049, 62.2697],
        [52.3740, 68.1767, 74.1632]])

In [306]:
targetData[0,:,:,0]

array([[61.53962369, 82.59228897, 62.86141313],
       [62.95067404, 65.1122754 , 55.53735889],
       [79.19024146, 77.40488797, 62.26973628],
       [52.37401402, 68.17673093, 74.16317807]])

In [311]:
(torch.FloatTensor(inputData) == torch.FloatTensor(inputData).permute(1,0,3,2).permute(1,0,3,2)).all()

tensor(1, dtype=torch.uint8)

In [323]:
getLoss(mockOutput, targetTTransformed, scaler)

tensor(15.1692)

In [326]:
np.sqrt(np.mean((inputData[...,0] - targetData[...,0])**2))

15.169173362390103